In [1]:
!pip install Faker
!pip install pandas 
!pip install tweepy
!pip install confluent_kafka

In [2]:
import time

from confluent_kafka import Producer
import json
import tweepy

producer = Producer(**{"bootstrap.servers": "kafka2:19092"})

In [3]:
consumer_key = "hK08N3zIwkJLrF6BVK52b3aS8"
consumer_secret = "8M22ppkZzb3sC9u4acUZK5IBr9nsLSByctTOoUI3Y5THEMRk7s"
access_token = "774855303321886720-JxdOM3ieqEVck6l0rYJmSIV3A412anW"
access_token_secret = "AoJgcvbtWmikzHaoOwH2rURrzoRwbBGgoV4oEASumNMzF"

In [4]:
# Words to track
WORDS = ['#coronavirus', '#COVID-19', '#COVID19', '#COVID'] #, '#SocialDistancing', '#pandemic']
raw_tweets_topic = "1_RAW_tweets"

In [5]:
class StreamListener(tweepy.StreamListener):
    # This is a class provided by tweepy to access the Twitter Streaming API.

    def on_connect(self):
        # Called initially to connect to the Streaming API
        print("You are now connected to the streaming API.")

    def on_error(self, status_code):
        # On error - if an error occurs, display the error / status code
        print("Error received in kafka producer " + repr(status_code))
        
        return True 
    
    def preprocess(self, tweetData):
        userID = '-'
        tweetText = '-'
        hashTags = []
        location_full_name = '-'
        created_at = '-'
        favoriteCount = 0
        reTweetCount = 0
        

        try:
            userID = tweetData["user"]["id"]
            
            if tweetData["truncated"] == True:
                tweetText = tweetData["extended_tweet"]["full_text"]
                hashTags = [ hTag["text"] for hTag in tweetData["extended_tweet"]["entities"]["hashtags"] ]
            else:
                tweetText = tweetData["text"]
                hashTags = [ hTag["text"] for hTag in tweetData["entities"]["hashtags"] ]

            if hashTags == []:
                return {}

            if "user" in tweetData and tweetData["user"] != None:
                location_full_name = tweetData["user"]["location"]
                
            created_at = tweetData["created_at"]
            favoriteCount = tweetData["favorite_count"]
            reTweetCount = tweetData["retweet_count"]
            

            rec = {'userID': userID, 
                   'tweetText': tweetText, 
                   'hashTags': hashTags, 
                   'location_full_name': location_full_name,
                   'favoriteCount': favoriteCount,
                   'reTweetCount': reTweetCount,
                   'created_at': created_at}

            return json.dumps(rec)

        except Exception as ex:
            print('Exception while parsing')
            print(str(ex))

    def on_data(self, data):
        """ This method is called whenever new data arrives from live stream.
        We asynchronously push this data to kafka queue"""
        try:
            parsed = json.loads(data)

            
            if "user" in parsed and "location" in parsed["user"]:
                if parsed["user"]["location"] != None:

                    processed = self.preprocess(parsed)

                    if isinstance(processed, str):
                        producer.produce(raw_tweets_topic, value=processed.encode('utf-8'))
                        producer.flush()

                        print(parsed["text"])

        except Exception as e:
            print("error! : " + str(e))
            return False #stop stream

        return True

    def on_timeout(self):
        return True

In [ ]:
# Create Auth object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Set up the listener. The 'wait_on_rate_limit=True' is needed to help with Twitter API rate limiting.
listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True, wait_on_rate_limit_notify=True, timeout=60, retry_delay=5, retry_count=10, retry_errors=set([401, 404, 500, 503])))
stream = tweepy.Stream(auth=auth, listener=listener)
print("Tracking: " + str(WORDS))
stream.filter(track=WORDS, languages = ['en'])

Tracking: ['#coronavirus', '#COVID-19', '#COVID19', '#COVID']
You are now connected to the streaming API.
Claiming that's it's all western media #propaganda to blame #China. Yet these reporters never cite how the pandemic… https://t.co/ELSEB3DtHx
RT @AntonyRobart: The debate over #VaccinePassports is heating up here in Canada- meanwhile in #Spain, the country’s top court has ruled ag…
RT @EuroELSO: Latest update about @EuroELSO survey on use of #ECMO in adults with #COVID19 respiratory &amp;/or cardiac failure in Europe:
🔹 nu…
#Most worrisome is a rise in severely ill patients, which reached a record high of 434." #SouthKorea #COVID19 https://t.co/0V5dtQPt7C
RT @DrAmbrishMithal: Start the day with some good news!
#VaccinationDrive picking up 
#India -Let's do our bit - convince at least 1 person…
@KavalAuthorActs Take Ivermectin tablets for best prevention and therapy of the #COVID19 virus and variants in non… https://t.co/rsvEaQMwHt
RT @SantaKlauSchwab: A lot of good info here for #CO

RT @Adam_Stirling: Apparently I had visitors earlier! 

Guys, vaccines aren’t a lie. The #covid19 vaccine is safe, effective, and it’s how…
RT @MsRonnyB: @annaclarity1 @abc730 Qld Premier @AnnastaciaMP specifically requested permission from #GladysSuperSpreader to extend border…
#Meghalaya government on Thursday announced the reopening of the state’s tourism sector from September 1 with relax… https://t.co/ngVmHAsjuc
@business @IndiaToday @ndtv @aajtak @ABPNews @CNN @BBCNews @WSJ @SEBI_India @Joker @cartoonnetwork @ComedyCentral… https://t.co/NtQtRsRKhe
Around half of patients discharged from hospital for Covid still suffer from at least one persistent symptom after… https://t.co/NKPSo6kixa
RT @portarican_RT: You blame Biden for deaths he can't control 8000 miles away, but won't blame red state governors for #COVID19 deaths the…
Some rich countries have ordered enough #COVID19 vaccine doses to vaccinate their entire country 3X over! Tell lead… https://t.co/p0nhbC6gKa
RT @portarican_RT:

RT @JohnKiehneForMo: Can you afford to take a few weeks off of work due to #COVID19 exposure? How about a few weeks in the hospital? Most p…
RT @htTweets: Zydus Cadila’s anti #Covid19 vaccine, ZyCov-D, the first approved for use on people as young as 12 in the country , is likely…
RT @htTweets: Maharashtra on Thursday breached the 5,000-mark for the second day in a row with 5,108 new #Covid19 cases and 159 deaths. Mum…
RT @CWM_en: #Taiwan's🇹🇼 #COVID19 fightback is going great guns!
Cooperation, trust, unity &amp; @audreyt's 1922 SMS Contact Tracing System are…
RT @Sambad_English: #BREAKING

#Odisha records 816 new #COVID19 cases, of which 106 were in 0-18 age group.

Infection tally rises to 10,04…
RT @NathanStall: All 34 Medical Officers of Health agree that #COVID19 vaccine passports are needed.

There is also overwhelming public sup…
RT @electroboyusa: These are extremely difficult and uncertain times between #COVID ravaging the country, the situation in #Afghanistan and…
RT @NC5Phi

...this facility three times. The workplace has been working at Alert Level 4 with PPE. The worker, in full PPE, di… https://t.co/muboGodHpF
RT @DrJohnB2: Two cases (30 y and 64 y) of venous sinus thrombosis following vaccination with the #AstraZeneca #CovidVaccine.
On the 4th da…
...who covers the facility has the opportunity to go in and vaccinate new people.
Will Northland being kept in Aler… https://t.co/Y52MwhA7tW
RT @xpresstn: #COVID19 India safety norms went for a toss in the #Puducherry Legislative Assembly during the address of Lt Governor Dr Tami…
RT @MeghUpdates: Kerala only state reporting over 1 lakh active #COVID19 cases: Govt

Pic credit: NDTV https://t.co/VopSc2JcG8
#UBC implements vaccine declaration and rapid testing for #COVID19.

#VancouverBC 🇨🇦

https://t.co/AVbzpZIjEE
RT @SouthAfricanASP: An image, both beautiful in its design and horrific in its content. Hundreds of millions of #COVID19 vaccines potentia…
RT @SamirShahMD: 💸😟Financial strain of #COVID19 hospitaliz

Pincode: 560011
COVISHIELD - Dose 2: 30 slots

Age: 18 and above  (All Ages)
Date: 27 Aug 2021
Fees: ₹780
Cloudnine… https://t.co/inCoFYPZpF
RT @ellymelly: Australians who simply wish to live as they have for their entire lives are being hunted down as #covid19 criminals.

When t…
RT @Devinde53997607: #GodMorningFriday
@SaintRampalJiM
@SatlokChannel

Diseases like cancer, AIDS are also cured by doing the devotion of G…
Really?
Did you pray for the 600,000 in 1 year dead from #DonaldTrump inaction on #COVID19  You know that is about… https://t.co/RH9TfEMcXG
The #EY Ireland Global Information Security Survey #GISS 2021 talks about the impact #COVID-19 had on Ireland’s cyb… https://t.co/oFtbu3k4rs
So weird. #covid #vaccine  https://t.co/yOGDEpTyio
@blkwomenshealth &amp; @JustDemocracy are hosting a discussion on the 58th anniversary of the “March on Washington”

WH… https://t.co/4NEzd6ff3z
RT @SkyNews: Award-winning radio presenter Lisa Shaw died due to rare complications from the AstraZe

RT @JoeMcFarland: So are we all just pretending that #COVID19 doesn't exist anymore? Not trying to be glib, just trying to understand where…
RT @htTweets: Zydus Cadila’s anti #Covid19 vaccine, ZyCov-D, the first approved for use on people as young as 12 in the country , is likely…
#Canada - Superior Court of Justice rules that jurors at an upcoming Ottawa murder trial must get jabbed against… https://t.co/6UEzvlylYK
#Nowplaying OMG - Usher  - Stream here https://t.co/YiXxSepm8x - Non Stop Hits from 70s to Today - Totally Commerci… https://t.co/VXTaZJMll1
RT @NathanStall: If preventable loss of life and suffering won’t convince the #Ontario government to implement #COVID19 vaccine certificate…
RT @mansukhmandviya: India achieves unprecedented milestone!

5️⃣0️⃣% of the eligible population inoculated with the first dose of #COVID19…
RT @GrittyNurse: My mom just called me and told me my 40 year old cousin died of #COVID19 . She lived here in #Ontario. She did not want to…
